# Reload cnmfe data to plot results

In [22]:
import matplotlib.pyplot as plt
import psutil
import numpy as np

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.utils.visualization import view_quilt

from decode_lab_code.calcium_imaging import calcium_imaging_utils

print(f"You have {psutil.cpu_count()} CPUs available in your current environment")
num_processors_to_use = 11

#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'cluster' in locals():  # 'locals' contains list of current local variables
    print('Closing previous cluster')
    cm.stop_server(dview=cluster)
print("Setting up new cluster")
dview, cluster, n_processes = cm.cluster.setup_cluster(backend='multiprocessing', 
                                                 n_processes=num_processors_to_use, 
                                                 ignore_preexisting=False)
print(f"Successfully set up cluster with {n_processes} processes")

You have 12 CPUs available in your current environment
Closing previous cluster
Setting up new cluster
Successfully set up cluster with 11 processes


In [19]:
# in case you are working from loaded data, recover the raw movie
mmap_name = r'/Users/js0403/miniscope/122A_session2_nwbfile/cnmfe_results.hdf5'
cnmfe_model = load_CNMF(mmap_name, 
                            n_processes=num_processors_to_use, 
                            dview=cluster)
correlation_image = cnmfe_model.estimates.Cn

In [27]:
if cnmfe_model.estimates.F_dff is None:
    print('Calculating estimates.F_dff')
    cnmfe_model.estimates.detrend_df_f(quantileMin=8, 
                                      frames_window=250,
                                      use_residuals=False);  # use denoised data
else:
    print("estimates.F_dff already defined")

Calculating estimates.F_dff


In [23]:
# in case you are working from loaded data, recover the raw movie
Yr, dims, num_frames = cm.load_memmap(cnmfe_model.mmap_file)
images = np.reshape(Yr.T, [num_frames] + list(dims), order='F')

In [24]:
# array of colors
from random import randint
colors = []
for i in range(1000):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

In [32]:
rois = cnmfe_model.estimates.coordinates
print(rois)

None


In [28]:
calcium_imaging_utils.plot_components(img=correlation_image, cnm=cnmfe_model, colors=colors, colorMap='hot')

TypeError: object of type 'NoneType' has no len()